This notebook 
-imports the .pth files version of the compressai models
-preprocess the images for the expected model like CHENG2020 takes special size of input image resolution
-run the inference on the images dataset
-get the average inference time of the dataset and get the quality metric of the reconstruction
-get the average inference time of the dataset too

In [1]:

import os
import torch
import time
from PIL import Image
import numpy as np
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [2]:
#checking the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
#importing the models from the compressai zoo
from compressai.zoo import models
#models avaialble are 
print("models are:",models.keys())

quality=4
metric="mse"#"ms-ssim" #"mse"
model_name="mbt2018"

net = models[model_name](quality=quality,metric=metric, pretrained=True).eval().to(device)
print("Model encoder architecture",model_name)

print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')

models are: dict_keys(['bmshj2018-factorized', 'bmshj2018-factorized-relu', 'bmshj2018-hyperprior', 'mbt2018-mean', 'mbt2018', 'cheng2020-anchor', 'cheng2020-attn', 'bmshj2018-hyperprior-vbr', 'mbt2018-mean-vbr', 'mbt2018-vbr', 'hrtzxf2022-pcc-rec', 'sfu2023-pcc-rec-pointnet', 'sfu2024-pcc-rec-pointnet2-ssg', 'ssf2020'])
Model encoder architecture mbt2018
Encoder Parameters: 14130467


In [ ]:
#trying the PyTorch exporting of the model
path_to_model_export=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\pytorch_models\cheng2020_anchor.pth"
#torch.save(net,path_to_model_export)
#print("model is exported")

#trying to load the model
model_imported=torch.load(path_to_model_export,weights_only=False)
model_imported.eval()

In [46]:
#single channel gray version testing 
#load the image as gray and resize and gray
image_=Image.open(path).convert("L")
image_=image.resize((1280,1280))
#make the numpy zeros tensor of the size
np_gray=np.zeros((3,1280,1280))
#assign the data to the first axis of the gray
np_gray[0]=np.array(image_)
np_gray=np_gray.transpose().astype(np.float32)
#check the gray matrix
#print(np_gray)
print("gray matrix shape:",np_gray.shape)

image_gray=transforms.ToTensor()(np_gray).unsqueeze(0).to(device)
print(image_gray.shape)
#check the type of the tensor
#print(image_gray)

NameError: name 'path' is not defined

In [3]:
def transform_3_channel_gray(image_path):
    image=Image.open(image_path) #already in the gray form and resized
    image_shape=list(np.array(image).shape) #get the image shape should be 2 elements
    np_gray=np.zeros((3,image_shape[0],image_shape[1]))
    np_gray[0]=np.array(image)#assign the first channel as the gray image
    #normalize the array
    np_gray=np_gray/255.0
    np_gray=np_gray.astype(np.float32)
    print("np_gray matrix is like:",np_gray.shape)
    #transform to the tensor
    image_gray=transforms.ToTensor()(np_gray).unsqueeze(0).permute(0, 2, 1, 3).to(device)
    print("final tensor shape:",image_gray.shape)
    return image_gray

def transform_color(image_path):
    image=Image.open(image_path) #already in the gray form and resized
    image=image.resize((1280,1280))
    image=transforms.ToTensor()(image).unsqueeze(0).to(device)
    #print("final tensor shape:",image.shape)
    return image
    
    


In [4]:

def run_inference(image_tensor):
    #run the forward pass computation 
    start=time.time()
    with torch.no_grad():
        #out_net = net.forward(x)
        out_net=net.forward(image_tensor)
    out_net['x_hat'].clamp_(0, 1)
    end=time.time()
    comp_time=round(end-start,2)
    print("Compression Time:",comp_time)
    
    output=out_net['x_hat']
    #print("output type is:",type(output))
    rec_net = transforms.ToPILImage()(out_net['x_hat'].squeeze())
    #output_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\PyTorch_inference\output.jpg"
    #rec_net.save(output_path,format="JPEG")
    #rec_net.show()
    return rec_net,comp_time

In [123]:
image_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50\dataset_1280x1280_gray\000000_left.png"
img_path_color=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50\000000_left.png"
image=Image.open(image_path)
#print(np.array(image).shape)

image_tensor=transform_color(img_path_color)
#print("shape:",image_tensor.shape)
#image_tensor=transform_3_channel_gray(image_path)
recon_img=run_inference(image_tensor)

#recon_img=recon_img.convert("L")
print(image_tensor.shape)
org_img=transforms.ToPILImage()(image_tensor.squeeze())
#print(type(org_img),org_img.size,org_img.mode)
#print(type(recon_img),recon_img.size,recon_img.mode)

#org_img=np.asarray(org_img)
#print("dtype:",org_img.dtype)

#recon_img=np.asarray(recon_img)
#print("dtype:",recon_img.dtype)

#calculate the metrics
print("SSIM:",round(ssim(np.asarray(org_img),np.asarray(recon_img),channel_axis=2),2))
print("PSNR:",round(psnr(np.asarray(org_img),np.asarray(recon_img)),2))

Compression Time: 0.36
output type is: <class 'torch.Tensor'>
torch.Size([1, 3, 1280, 1280])
SSIM: 0.92
PSNR: 30.94


In [7]:
print("models are:",models.keys())

models are: dict_keys(['bmshj2018-factorized', 'bmshj2018-factorized-relu', 'bmshj2018-hyperprior', 'mbt2018-mean', 'mbt2018', 'cheng2020-anchor', 'cheng2020-attn', 'bmshj2018-hyperprior-vbr', 'mbt2018-mean-vbr', 'mbt2018-vbr', 'hrtzxf2022-pcc-rec', 'sfu2023-pcc-rec-pointnet', 'sfu2024-pcc-rec-pointnet2-ssg', 'ssf2020'])


In [8]:
quality=8
metric="mse"#"ms-ssim" #"mse"
model_name="bmshj2018-factorized-relu"

net = models[model_name](quality=quality,metric=metric, pretrained=True).eval().to(device)
print("Model encoder architecture",model_name)

print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')
export_name=model_name+"_"+str(metric)+"_"+str(quality)

def run_inference_on_dataset(dataset_path,model_name,export_path_base,export_name):

    dataset_export_path=os.path.join(export_path_base,export_name)
    print("export directory name:",export_name)

    try:
        os.makedirs(dataset_export_path)
    except:
        print("Directory already exists")

    #load the images 
    files=os.listdir(dataset_path)
    images=[]
    #assuming the dataset is only of PNG files

    comp_time=[]
    ssim_lst=[]
    psnr_lst=[]
    transform_time_=[]
    for file in files:
        if file.lower().endswith(".png"):
            images.append(file)
            
    print("images:",images)
    for img in images:
        start=time.time()
        image_tensor=transform_color(os.path.join(dataset_path,img)) #only resize
        end=time.time()
        transform_time=round(end-start,2)
    
        
        recon_img,comp_time_=run_inference(image_tensor)
        img_export_path=os.path.join(dataset_export_path,img.split('.')[0])
        recon_img.save(img_export_path+".jpg",format="JPEG",quality=60)
        
        
        
        
        start=time.time()
        org_img=transforms.ToPILImage()(image_tensor.squeeze())
        end=time.time()
        transform_back_time=round(end-start,2)
        
        
        psnr_=round(psnr(np.asarray(org_img),np.asarray(recon_img)),2)
        ssim_=round(ssim(np.asarray(org_img),np.asarray(recon_img),channel_axis=2),2)
        
        
        
        #print("loop time:",loop_time,"seconds")
        comp_time.append(comp_time_)
        ssim_lst.append(ssim_)
        psnr_lst.append(psnr_)
        transform_time_.append(transform_time+transform_back_time)
        
    print("For the model_name",model_name)
    print("Avg_comp_time",np.array(comp_time).mean())
    print("Avg_ssim",np.array(ssim_lst).mean())
    print("Avg_psnr",np.array(psnr_lst).mean())
    print("Avg_total process time per image",np.array(transform_time_).mean())
    # Create a list of data to be exported
    data = [
        ("Model Name", model_name),
        ("Avg. Comp. Time", np.array(comp_time).mean()),
        ("Avg. SSIM", np.array(ssim_lst).mean()),
        ("Avg. PSNR", np.array(psnr_lst).mean()),
        ("Avg. Total Process Time per Image", np.array(transform_time_).mean())
    ]
    
    # Write the data to a text file
    results_export=os.path.join(dataset_export_path,export_name+".txt")
    with open(results_export, "w") as f:
        for item in data:
            f.write("{}: {}\n".format(item[0], item[1]))
        
        

    


dataset_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50\data_20"
export_path_base=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50\data_20"
#run_inference_on_dataset(dataset_path,model_name,export_path_base,export_name)


for model_name in list(models.keys()):
    for quality in [4,6,8]:
        for metric in ["mse","ms-ssim"]:
            export_name=model_name+"_"+str(metric)+"_"+str(quality)
            try:
                net = models[model_name](quality=quality,metric=metric, pretrained=True).eval().to(device)
                print(f'Encoder Parameters: {sum(p.numel() for p in net.parameters())}')
                
            except:
                print("Failed:",export_name)
                continue
            run_inference_on_dataset(dataset_path,model_name,export_path_base,export_name)





Model encoder architecture bmshj2018-factorized-relu
Encoder Parameters: 7030531
Encoder Parameters: 2998147
export directory name: bmshj2018-factorized_mse_4
images: ['000000_left.png', '000001_left.png', '000002_left.png', '000003_left.png', '000004_left.png', '000005_left.png', '000006_left.png', '000007_left.png', '000008_left.png', '000009_left.png', '000010_left.png', '000011_left.png', '000012_left.png', '000013_left.png', '000014_left.png', '000015_left.png', '000016_left.png', '000017_left.png', '000018_left.png', '000019_left.png']
Compression Time: 0.34
Compression Time: 0.07
Compression Time: 0.07
Compression Time: 0.05
Compression Time: 0.11
Compression Time: 0.07
Compression Time: 0.04
Compression Time: 0.04
Compression Time: 0.05
Compression Time: 0.07
Compression Time: 0.05
Compression Time: 0.06
Compression Time: 0.05
Compression Time: 0.04
Compression Time: 0.06
Compression Time: 0.04
Compression Time: 0.06
Compression Time: 0.04
Compression Time: 0.05
Compression Tim

RuntimeError: Invalid number of frames: 1.

In [116]:

def transform_dataset(dataset_path,dataset_export_path_base,resize_shape):

    export_dir_name="dataset"+"_"+str(resize_shape[0])+"x"+str(resize_shape[1])+"_gray"
    dataset_export_path=os.path.join(dataset_export_path_base,export_dir_name)
    print("export directory name:",export_dir_name)

    try:
        os.makedirs(dataset_export_path)
    except:
        print("Directory already exists")
    
    files=os.listdir(dataset_path)
    images=[]
    #assuming the dataset is only of PNG files
    for file in files:
        if file.lower().endswith(".png"):
            images.append(file)
            
    print("images:",images)
    for img in images:
        image=Image.open(os.path.join(dataset_path,img)).convert("L") #convert to gray
        image=image.resize(resize_shape) #assuming the resize shape is the tuple(width,height)
        image.save(os.path.join(dataset_export_path,img))
        #print("image:",img,"exported")


#Running the function to make the gray dataset of the following shape
resize_shape=[1280,1280]
dataset_path=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50" #this is the 50 PNG images color dataset path
dataset_export_path_base=r"C:\Swapnil\Narrowband_DRONE\Image_compression_code\Final_DL_compession_September_24\Dataset_50"
transform_dataset(dataset_path,dataset_export_path_base,resize_shape)


export directory name: dataset_1280x1280_gray
Directory already exists
images: ['000000_left.png', '000001_left.png', '000002_left.png', '000003_left.png', '000004_left.png', '000005_left.png', '000006_left.png', '000007_left.png', '000008_left.png', '000009_left.png', '000010_left.png', '000011_left.png', '000012_left.png', '000013_left.png', '000014_left.png', '000015_left.png', '000016_left.png', '000017_left.png', '000018_left.png', '000019_left.png', '000020_left.png', '000021_left.png', '000022_left.png', '000023_left.png', '000024_left.png', '000025_left.png', '000026_left.png', '000027_left.png', '000028_left.png', '000029_left.png', '000030_left.png', '000031_left.png', '000032_left.png', '000033_left.png', '000034_left.png', '000035_left.png', '000036_left.png', '000037_left.png', '000038_left.png', '000039_left.png', '000040_left.png', '000041_left.png', '000042_left.png', '000043_left.png', '000044_left.png', '000045_left.png', '000046_left.png', '000047_left.png', '000048_

In [ ]:
#make the numpy zeros tensor of the size
        #np_gray=np.zeros((3,resize_shape[0],resize_shape[1]))
        #assign the data to the first axis of the gray
        #np_gray[0]=np.array(image)
        #np_gray=np_gray.astype(np.uint8)
        #np_gray=np_gray.transpose()
        #check the gray matrix
        #print(np_gray)
        #print("gray matrix shape:",np_gray.shape)
        #exporting the image to the disc
        #final_image=Image.fromarray(np_gray) #save as grayscale image
        #final_image.save(os.path.join(dataset_export_path,img))

#use this logic while running the inference from the image